In [ ]:
import numpy as np
import gym
from numpy.random import choice
import random
from scipy.interpolate import interp1d

env = gym.make("Taxi-v2")

In [ ]:
n_actions = env.action_space.n
n_states = env.observation_space.n
Q = np.zeros([n_states, n_actions])

y = 0.99
k = 10000.0

In [ ]:
print(Q.shape)
print(Q)

In [ ]:
def next_action(actions, e):
    n = len(actions)
    
    if random.random() < e:
        return random.randint(0, n-1)
    else:
        return np.argmax(actions)

In [ ]:
s = env.reset()
for i in range(1000):
    a = env.action_space.sample()
    s, r, done, info = env.step(a)
    env.render()
    print(r)
    print("")
    
    if done:
        break

In [ ]:
r_total = 0
e_fun = interp1d([0, 40000], [1, 0.01], bounds_error=False, fill_value=0.01)
learning_rate_f = lambda t: k / (k + t)
global_step = 0

for t in range(200000):
    s = env.reset()
    
    while True:
        global_step += 1
        learning_rate = learning_rate_f(global_step)
        e = e_fun(global_step)
        
        a = next_action(Q[s], e)

        s1, r, done, info = env.step(a)
        r_total += r

        Q[s, a] += learning_rate * (r + y * np.max(Q[s1]) - Q[s, a])
        
        s = s1
        
        if done:
            break
            
    if t % 500 == 0:
        print(r_total / 500.0)
        r_total = 0

In [ ]:
Q

In [ ]:
s = env.reset()
for i in range(1000):
    a = next_action(Q[s], 0)
    s, r, done, info = env.step(a)
    env.render()
    print(r)
    print("")
    
    if done:
        break